In [1]:
import numpy as np
import pandas as pd
import requests

# Sensitivity to eutrophication

# Part 2: Waterbody properties

Scraping additional information for each waterbody from Vann-Nett. See e-mail from Sonja K received 21.01.2025 for details.

## 1. Waterbodies of interest

In [2]:
# Read WB list from MDir
xl_path = r"../data/mdir_waterbodies_list.xlsx"
wb_df = pd.read_excel(xl_path)

# Strip leading and trailing whitespace
wb_df["VannforekomstID"] = wb_df["VannforekomstID"].str.strip()

# Check unique
print("The following waterbodies are duplicated. Only the last will be kept:")
display(wb_df[wb_df.duplicated("VannforekomstID", keep=False)])
wb_df.drop_duplicates("VannforekomstID", keep="last", inplace=True)
wb_df = wb_df.dropna(subset="VannforekomstID").reset_index(drop=True)

The following waterbodies are duplicated. Only the last will be kept:


,VannforekomstID,Id,Navn.vannforekomst,Breddegrad,Lengdegrad,Zero,Areal (km2),Type,Opphold.bunnvann,NIVA.data,oppdrag
141,050-106-R,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,Hovedoppdrag
142,050-105-R,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,Hovedoppdrag
153,050-105-R,R01,Bjoreio overføring fra Sysenvatnet - dam Tveito,60.4217,7.2528,0.0,17.369,R,NaN,NaN,NaN
154,050-106-R,R02,Isdølo Isdalsvatnet - Bjoreio,60.4302,7.2762,0.0,3.356,R,NaN,Vannforekomsten oppstrøms er med i referanseel...,NaN


In [3]:
# Get details from Vann-Nett
wb_list = wb_df["VannforekomstID"].tolist()
sal_list = []
type_list = []

for wb_id in wb_list:
    url = f"https://vann-nett.no/service/waterbody/{wb_id}"
    response = requests.get(url)
    if response.status_code != 200:
        response.raise_for_status()
    data = response.json()
    
    try:
        sal = data["waterType"]["typeParameters"]["salinity"]["text"]
    except KeyError:
        sal = np.nan

    try:
        water_type = data["waterType"]["nationalWaterType"]
    except KeyError:
        water_type = np.nan

    sal_list.append(sal)
    type_list.append(water_type)
    
wb_df["national_water_type"] = type_list
wb_df["salinity_class"] = sal_list

out_xl_path = r"../data/mdir_waterbodies_with_type.xlsx"
wb_df.to_excel(out_xl_path, index=False)